In [1]:
# Import required packages
import os
import shutil

import numpy as np
import sklearn.utils as sku

import Config as conf
import CSV as csv

In [2]:
# Import CSV data
csi, label, size = csv.getWindows()

Loading Dataset/csi_1123_fall_a_200_1x3_amp.csv and Dataset/label_1123_fall_a_200_1x3_amp.csv ...
 - PPS determined: 200
 - COL determined: 90
 - ROW determined: 120
 - SLIDE determined: 1
 - RECOG determined: 90
Loading Dataset/csi_1123_fall_b_200_1x3_amp.csv and Dataset/label_1123_fall_b_200_1x3_amp.csv ...
Loading Dataset/csi_1123_fall_c_200_1x3_amp.csv and Dataset/label_1123_fall_c_200_1x3_amp.csv ...
Loading Dataset/csi_1123_fall_d_1_200_1x3_amp.csv and Dataset/label_1123_fall_d_1_200_1x3_amp.csv ...
Loading Dataset/csi_noise_200_1x3_amp.csv and Dataset/label_noise_200_1x3_amp.csv ...
54972 windows may produced at maximum.
Creating windows from Dataset/csi_1123_fall_a_200_1x3_amp.csv and Dataset/label_1123_fall_a_200_1x3_amp.csv ...
Creating windows from Dataset/csi_1123_fall_b_200_1x3_amp.csv and Dataset/label_1123_fall_b_200_1x3_amp.csv ...
Creating windows from Dataset/csi_1123_fall_c_200_1x3_amp.csv and Dataset/label_1123_fall_c_200_1x3_amp.csv ...
Creating windows from Datase

In [3]:
# Import Keras
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.callbacks as kc
import tensorflow.keras.layers as kl
import tensorflow.keras.models as km
import tensorflow.keras.optimizers as ko
import tensorflow.keras.utils as ku

# Set CUDA (use what gpu?) -- comment this if use all GPUs
os.environ["CUDA_VISIBLE_DEVICES"]="0,2,3"

# Print tensorflow version
print("Tensorflow:", tf.__version__)
print("Keras:", keras.__version__)

Tensorflow: 2.0.0
Keras: 2.2.4-tf


In [4]:
# Setup Keras LSTM Model
model = None
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    adam = ko.Adam(learning_rate=conf.LEARNING_RATE, amsgrad=True)
    lstm = kl.LSTM(
        2000,
        unit_forget_bias=True,
        input_shape=(size[0], size[1]))
    lstm.add_loss(lambda: 1e-8)
    model = km.Sequential()
    model.add(lstm)
    model.add(kl.Dense(conf.ACTION_CNT, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=["accuracy"]
    )
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2000)              16728000  
_________________________________________________________________
dense (Dense)                (None, 5)                 10005     
Total params: 16,738,005
Trainable params: 16,738,005
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Check output directory and prepare tensorboard
if os.path.exists(conf.OUTPUT_DIR):
    shutil.rmtree(conf.OUTPUT_DIR)
os.makedirs(conf.OUTPUT_DIR)
if os.path.exists(conf.LOG_DIR):
    shutil.rmtree(conf.LOG_DIR)
os.makedirs(conf.LOG_DIR)
tensorboard = kc.TensorBoard(
    log_dir=conf.LOG_DIR,
    write_graph=True,
    write_images=True,
    update_freq=10)
print(
    "Your tensorboard command is:"
)
print("    tensorboard --logdir=" + conf.LOG_DIR)
print("Keras checkpoints and final result will be saved in here:")
print("    " + conf.OUTPUT_DIR)

Your tensorboard command is:
    tensorboard --logdir=./Log_LR0.002_B256_K10_CP2048/
Keras checkpoints and final result will be saved in here:
    ./Output_LR0.002_B256_K10_CP2048/


In [6]:
# Run KFold
xx, yy = sku.shuffle(csi, label, random_state=0)
for i in range(conf.KFOLD):
    # Roll the data
    xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = xx[int(len(xx) / conf.KFOLD):]
    yTrain = yy[int(len(yy) / conf.KFOLD):]
    xEval = xx[:int(len(xx) / conf.KFOLD)]
    yEval = yy[:int(len(yy) / conf.KFOLD)]

    # If there exists only one action, convert Y to binary form
    if yEval.shape[1] == 1:
        yTrain = ku.to_categorical(yTrain)
        yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        conf.OUTPUT_DIR + "Checkpoint_K" + str(i + 1) + "_EPOCH{epoch}.h5",
        save_freq=conf.CP_FREQ,
        verbose=1)

    # Fit model (learn)
    print(str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) + " th.")
    model.fit(
        xTrain,
        yTrain,
        epochs=conf.EPOCH_CNT,
        batch_size=conf.BATCH_SIZE,
        shuffle=True,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval),
        validation_freq=conf.EVAL_FREQ,
        use_multiprocessing=True)
print("Epoch completed!")

1 th fitting started. Endpoint is 10 th.
Train on 4956 samples, validate on 550 samples
Epoch 1/64
1792/4956 [=========>....................] - ETA: 41s - loss: 4.4893 - accuracy: 0.2394
Epoch 00001: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH1.h5
3840/4956 [======================>.......] - ETA: 11s - loss: 3.0975 - accuracy: 0.2536
Epoch 00001: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH1.h5
4956/4956 [==============================] - 50s 10ms/sample - loss: 2.7589 - accuracy: 0.2514
Epoch 2/64
1024/4956 [=====>........................] - ETA: 30s - loss: 1.3919 - accuracy: 0.5000
Epoch 00002: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH2.h5
3072/4956 [=================>............] - ETA: 14s - loss: 1.2980 - accuracy: 0.5036
Epoch 00002: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH2.h5
4956/4956 [==============================] - 44s 9ms/sample - loss: 1.2276 - accuracy: 0.5200 - val

4608/4956 [==========================>...] - ETA: 2s - loss: 0.1924 - accuracy: 0.9447
Epoch 00016: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH16.h5
4956/4956 [==============================] - 43s 9ms/sample - loss: 0.1895 - accuracy: 0.9461 - val_loss: 0.1249 - val_accuracy: 0.9727
Epoch 17/64
1792/4956 [=========>....................] - ETA: 24s - loss: 0.1731 - accuracy: 0.9503
Epoch 00017: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH17.h5
3840/4956 [======================>.......] - ETA: 8s - loss: 0.1738 - accuracy: 0.9469 
Epoch 00017: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH17.h5
4956/4956 [==============================] - 40s 8ms/sample - loss: 0.1689 - accuracy: 0.9485
Epoch 18/64
1024/4956 [=====>........................] - ETA: 30s - loss: 0.1511 - accuracy: 0.9521
Epoch 00018: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH18.h5
3072/4956 [=================>............] - ET

 512/4956 [==>...........................] - ETA: 34s - loss: 0.0214 - accuracy: 0.9961
Epoch 00032: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH32.h5
2560/4956 [==============>...............] - ETA: 19s - loss: 0.0220 - accuracy: 0.9949
Epoch 00032: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH32.h5
4608/4956 [==========================>...] - ETA: 2s - loss: 0.0262 - accuracy: 0.9937
Epoch 00032: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH32.h5
4956/4956 [==============================] - 42s 9ms/sample - loss: 0.0271 - accuracy: 0.9931 - val_loss: 0.1377 - val_accuracy: 0.9636
Epoch 33/64
1792/4956 [=========>....................] - ETA: 24s - loss: 0.0540 - accuracy: 0.9866
Epoch 00033: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH33.h5
3840/4956 [======================>.......] - ETA: 8s - loss: 0.0432 - accuracy: 0.9891 
Epoch 00033: saving model to ./Output_LR0.002_B256_K10_CP2048/Che

3328/4956 [===================>..........] - ETA: 12s - loss: 0.0240 - accuracy: 0.9955
Epoch 00047: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH47.h5
4956/4956 [==============================] - 40s 8ms/sample - loss: 0.0263 - accuracy: 0.9941
Epoch 48/64
 512/4956 [==>...........................] - ETA: 34s - loss: 0.0227 - accuracy: 0.9961
Epoch 00048: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH48.h5
2560/4956 [==============>...............] - ETA: 19s - loss: 0.0255 - accuracy: 0.9949
Epoch 00048: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH48.h5
4608/4956 [==========================>...] - ETA: 2s - loss: 0.0225 - accuracy: 0.9950
Epoch 00048: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH48.h5
4956/4956 [==============================] - 42s 9ms/sample - loss: 0.0249 - accuracy: 0.9939 - val_loss: 0.0286 - val_accuracy: 0.9891
Epoch 49/64
1792/4956 [=========>....................] - ET

4956/4956 [==============================] - 43s 9ms/sample - loss: 0.0573 - accuracy: 0.9861 - val_loss: 0.0469 - val_accuracy: 0.9891
Epoch 63/64
1280/4956 [======>.......................] - ETA: 28s - loss: 0.0481 - accuracy: 0.9875
Epoch 00063: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH63.h5
3328/4956 [===================>..........] - ETA: 12s - loss: 0.0453 - accuracy: 0.9883
Epoch 00063: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH63.h5
4956/4956 [==============================] - 40s 8ms/sample - loss: 0.0428 - accuracy: 0.9893
Epoch 64/64
 512/4956 [==>...........................] - ETA: 34s - loss: 0.0559 - accuracy: 0.9844
Epoch 00064: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH64.h5
2560/4956 [==============>...............] - ETA: 18s - loss: 0.0453 - accuracy: 0.9895
Epoch 00064: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K1_EPOCH64.h5
4608/4956 [==========================>...] - E

...

4608/4956 [==========================>...] - ETA: 1s - loss: 7.7841e-04 - accuracy: 1.0000
Epoch 00056: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K9_EPOCH56.h5
4956/4956 [==============================] - 28s 6ms/sample - loss: 7.8095e-04 - accuracy: 1.0000 - val_loss: 7.3220e-04 - val_accuracy: 1.0000
Epoch 57/64
1792/4956 [=========>....................] - ETA: 18s - loss: 7.7531e-04 - accuracy: 1.0000
Epoch 00057: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K9_EPOCH57.h5
3840/4956 [======================>.......] - ETA: 6s - loss: 7.8001e-04 - accuracy: 1.0000
Epoch 00057: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K9_EPOCH57.h5
4956/4956 [==============================] - 29s 6ms/sample - loss: 7.7606e-04 - accuracy: 1.0000
Epoch 58/64
1024/4956 [=====>........................] - ETA: 22s - loss: 7.6115e-04 - accuracy: 1.0000
Epoch 00058: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K9_EPOCH58.h5
3072/4956 [=========

3328/4956 [===================>..........] - ETA: 9s - loss: 7.1783e-04 - accuracy: 1.0000 
Epoch 00007: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH7.h5
4956/4956 [==============================] - 29s 6ms/sample - loss: 7.2662e-04 - accuracy: 1.0000
Epoch 8/64
 512/4956 [==>...........................] - ETA: 25s - loss: 7.6303e-04 - accuracy: 1.0000
Epoch 00008: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH8.h5
2560/4956 [==============>...............] - ETA: 14s - loss: 7.4320e-04 - accuracy: 1.0000
Epoch 00008: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH8.h5
4608/4956 [==========================>...] - ETA: 2s - loss: 7.2345e-04 - accuracy: 1.0000
Epoch 00008: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH8.h5
4956/4956 [==============================] - 34s 7ms/sample - loss: 7.2250e-04 - accuracy: 1.0000 - val_loss: 6.6437e-04 - val_accuracy: 1.0000
Epoch 9/64
1792/4956 [=========>

4096/4956 [=======================>......] - ETA: 4s - loss: 6.8760e-04 - accuracy: 1.0000
Epoch 00022: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH22.h5
4956/4956 [==============================] - 31s 6ms/sample - loss: 6.7702e-04 - accuracy: 1.0000 - val_loss: 6.2407e-04 - val_accuracy: 1.0000
Epoch 23/64
1280/4956 [======>.......................] - ETA: 20s - loss: 7.4082e-04 - accuracy: 1.0000
Epoch 00023: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH23.h5
3328/4956 [===================>..........] - ETA: 9s - loss: 6.7668e-04 - accuracy: 1.0000 
Epoch 00023: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH23.h5
4956/4956 [==============================] - 29s 6ms/sample - loss: 6.7364e-04 - accuracy: 1.0000
Epoch 24/64
 512/4956 [==>...........................] - ETA: 31s - loss: 6.4566e-04 - accuracy: 1.0000
Epoch 00024: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH24.h5
2560/4956 [====

2816/4956 [================>.............] - ETA: 14s - loss: 6.1947e-04 - accuracy: 1.0000
Epoch 00037: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH37.h5
4956/4956 [==============================] - 31s 6ms/sample - loss: 6.2789e-04 - accuracy: 1.0000
Epoch 38/64

Epoch 00038: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH38.h5
2048/4956 [===========>..................] - ETA: 15s - loss: 6.3325e-04 - accuracy: 1.0000
Epoch 00038: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH38.h5
4096/4956 [=======================>......] - ETA: 4s - loss: 6.3325e-04 - accuracy: 1.0000
Epoch 00038: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH38.h5
4956/4956 [==============================] - 30s 6ms/sample - loss: 6.2419e-04 - accuracy: 1.0000 - val_loss: 5.7801e-04 - val_accuracy: 1.0000
Epoch 39/64
1280/4956 [======>.......................] - ETA: 20s - loss: 6.0868e-04 - accuracy: 1.0000
Epoch 00039: s

3584/4956 [====================>.........] - ETA: 7s - loss: 5.6987e-04 - accuracy: 1.0000
Epoch 00052: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH52.h5
4956/4956 [==============================] - 29s 6ms/sample - loss: 5.6853e-04 - accuracy: 1.0000 - val_loss: 5.3083e-04 - val_accuracy: 1.0000
Epoch 53/64
 768/4956 [===>..........................] - ETA: 21s - loss: 5.8171e-04 - accuracy: 1.0000
Epoch 00053: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH53.h5
2816/4956 [================>.............] - ETA: 13s - loss: 5.4987e-04 - accuracy: 1.0000
Epoch 00053: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH53.h5
4956/4956 [==============================] - 30s 6ms/sample - loss: 5.6506e-04 - accuracy: 1.0000
Epoch 54/64

Epoch 00054: saving model to ./Output_LR0.002_B256_K10_CP2048/Checkpoint_K10_EPOCH54.h5
2048/4956 [===========>..................] - ETA: 15s - loss: 5.4852e-04 - accuracy: 1.0000
Epoch 00054: s

In [8]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(conf.OUTPUT_DIR + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(conf.OUTPUT_DIR + "model.json", "w") as json:
    json.write(modelJSON)
model.save(conf.OUTPUT_DIR + "model.h5")
print('Model saved!')

Saving model & model information...
Model saved!


In [ ]:
# Finished